In [24]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [25]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
PostCode = df["Postcode"].tolist()
Borough = df["Borough"].tolist()
Neighbourhood=df["Neighbourhood"].tolist()
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [28]:
df=df[df.Borough != 'Not assigned']

In [36]:
df['Postcode'] = df['Postcode'].astype(str)
df['Neighbourhood'] = df['Neighbourhood'].astype(str)
df = df.groupby(['Postcode','Borough'], sort=False).agg( ','.join)
df.head()



,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Queen's Park


In [47]:
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])

KeyError: 'Borough'

In [48]:
df.head(10)

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Queen's Park
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,"Rouge,Malvern"
M3B,North York,Don Mills North
M4B,East York,"Woodbine Gardens,Parkview Hill"


## I cleaned dataframe and dropped unnecessary values, grouped by Postcode, so size is much smaller.

In [39]:
df.shape

(103, 1)

In [40]:
geodata=pd.read_csv('http://cocl.us/Geospatial_data')
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [58]:
df_all=pd.merge(geodata,df,left_on='Postal Code',right_on='Postcode',how='outer')


In [59]:
df_all.head(10)

,Postal Code,Latitude,Longitude,Neighbourhood
0,M1B,43.806686,-79.194353,"Rouge,Malvern"
1,M1C,43.784535,-79.160497,"Highland Creek,Rouge Hill,Port Union"
2,M1E,43.763573,-79.188711,"Guildwood,Morningside,West Hill"
3,M1G,43.770992,-79.216917,Woburn
4,M1H,43.773136,-79.239476,Cedarbrae
5,M1J,43.744734,-79.239476,Scarborough Village
6,M1K,43.727929,-79.262029,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,43.711112,-79.284577,"Clairlea,Golden Mile,Oakridge"
8,M1M,43.716316,-79.239476,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,43.692657,-79.264848,"Birch Cliff,Cliffside West"


## I am going to make a map out of data that I cleaned and processed

In [64]:

import matplotlib.pyplot as plt
import folium
df_all = folium.Map(location=[43.6532, 79.3832], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_all['Latitude'], df_all['Longitude'], df_all['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
df_all

TypeError: 'Map' object is not subscriptable